In [1]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pygmt
import os
from scipy.spatial.distance import pdist
import time
from datetime import datetime
from multiprocessing import Pool, cpu_count
from scipy.spatial import KDTree
from scipy import interpolate
#  ========================================================================================================
def project_and_get_distances(points, reference_point):
    from pyproj import Transformer
    transformer = Transformer.from_crs("EPSG:4326", "EPSG:32648", always_xy=True)
    
    # Project all points
    x, y = transformer.transform(points[:, 0], points[:, 1])
    x0, y0 = transformer.transform(reference_point[0], reference_point[1])
    xy = np.column_stack((x, y))
    ref_xy = np.array([x0, y0])
    distances = np.linalg.norm(xy - ref_xy, axis=1)
    return distances
def dist2ref(df,ref_point,keylong='Longitude',keylat='Latitude'):
    from pyproj import Geod
    '''
    calculate the distance from the spicies coordinate to reference point
    Input is the data frame, the distance is a new column
    '''
    # Define geodetic system (WGS84)
    geod = Geod(ellps='WGS84')
    # Compute forward and back azimuths and distance
    # Function to compute distance
    def compute_distance(row):
        az12, az21, dist = geod.inv(ref_point[0], ref_point[1],row[keylong], row[keylat])
        az_rad = np.deg2rad(az12)
        dx = dist * np.sin(az_rad)  # East-west
        dy = dist * np.cos(az_rad)  # North-south
        return pd.Series([dist, dx, dy], index=['distance','d2x', 'd2y'])  # in meters
    # Apply row-wise
    df[['distance','d2x', 'd2y']] = df.apply(compute_distance, axis=1)
    return df  


def ripley_k_border_corrected(points, reference_point, max_radius, step=1.0, window=None, inner_window=None):
    """
    Ripley's K with border (reduced sample) correction, using predefined inner usable area.

    Args:
        points (ndarray): Nx2 array of point coordinates.
        max_radius (float): Maximum radius to evaluate K.
        step (float): Distance step size.
        window (tuple): (xmin, xmax, ymin, ymax), the full sampling area.
        inner_window (tuple): (xmin, xmax, ymin, ymax), the inner area where circles of radius r fit entirely.

    Returns:
        radii (ndarray): Radii at which K(r) is evaluated.
        K_values (ndarray): Border-corrected K values.
    """
    n = len(points)
    if n < 2:
        raise ValueError("Need at least two points.")

    # Determine full window (outer box)
    if window is None:
        xmin, ymin = np.min(points, axis=0)
        xmax, ymax = np.max(points, axis=0)
    else:
        xmin, xmax, ymin, ymax = window

    # Determine inner usable area
    if inner_window is None:
        # Use dynamic check (original way)
        use_dynamic = True
    else:
        ixmin, ixmax, iymin, iymax = inner_window
        use_dynamic = False

    area = (xmax - xmin) * (ymax - ymin)
    density = n / area
    
    tree = KDTree(points)
    radii = np.arange(step, max_radius + step, step)
    K_values = np.zeros_like(radii)

    for i, r in enumerate(radii):
        if use_dynamic: # border correction
            # Dynamically choose valid points
            valid_points = [p for p in points if (
                p[0] - r >= xmin and p[0] + r <= xmax and
                p[1] - r >= ymin and p[1] + r <= ymax)]
        else: # no border correction
            # Use inner window provided
            valid_points = [p for p in points if (
                p[0] >= ixmin and p[0] <= ixmax and
                p[1] >= iymin and p[1] <= iymax)]

        valid_points = np.array(valid_points)
        if len(valid_points) < 2:
            continue

        count = tree.query_ball_point(reference_point, r, return_length=True)
        # total_neighbors = np.sum(counts) - len(valid_points)
        K_values[i] = (count - 1) / density

    return radii, K_values
    
    
def ripley_k(points, reference_point, radius, area):
    """
    Compute the Ripley-K function for a given reference point.
    + Input: 
    """
    # distances = np.linalg.norm(points - reference_point, axis=1)
    distances = project_and_get_distances(points, reference_point)
    count_within_radius = np.sum(distances <= radius) - 1  # exclude self
    K = (area / len(points)) * count_within_radius
    return K
def ripley_k_full(points, radius, area):
    from scipy.spatial.distance import pdist, squareform
    '''
    This calculate the ripley-K for full data set, not for single point
    '''
    n = len(points)
    dist_matrix = squareform(pdist(points))
    count = np.sum((dist_matrix <= radius)) - n  # subtract self-counts
    K = (area / (n * (n - 1))) * count
    return K
    
def derivative_ripley_k(points, reference_point, radius, area, delta_r=1e-5):
    """
    Compute the derivative of the Ripley-K function at a reference point with respect to the radius.
    """
    K_r = ripley_k(points, reference_point, radius, area)
    K_r_plus_delta = ripley_k(points, reference_point, radius + delta_r, area)
    dK_dr = ((K_r_plus_delta - K_r) / delta_r)
    # print("> Radius:",radius)
    if radius == 0:
        gr = 0
        Lr = 0
    else:
        gr = dK_dr/(2*np.pi*radius)
        Lr = np.sqrt(K_r/np.pi) - radius
    return gr, Lr

# Helper function to apply to each point
def compute_gr_for_point(args):
    '''
    Parallel computing for this GR
    '''
    point, radius, area, step, pointxy = args
    # print(point)
    refpoint=[point[1][2],point[1][3]]
    #
    # radii, gr, Lr = derivative_ripley_k_range(points_global_only_coords, point, radius, area, step, 0.1)
    radii, Kr = ripley_k_border_corrected(pointxy, refpoint, radius, step=1.0, window=None, inner_window=None)
    # ---------------------------------------------------------
    # now calculate derivative dK_dr
    dK_dr = np.gradient(Kr,radii)
    gr = dK_dr/(2 * np.pi * radii); gr[gr < 0] = 0;
    Lr = np.sqrt(Kr / np.pi) - radii;
    # ---------------------------------------------------------
    return pd.DataFrame({
        'Point': np.full(len(radii), point[0]),
        'Longitude': np.full(len(radii), point[1][0]),
        'Latitude': np.full(len(radii), point[1][1]),
        'radius_m': radii,
        'gr': gr,
        'Lr': Lr
    })
    # ---------------------------------------------------------
#  ========================================================================================================
if __name__ == "__main__":
    while True:
        answer = input("This code run for 2 spices - so all data will be select. RUN? (yes/no): ").strip().lower()
        if answer in ["yes", "no"]:
            break
        print("Please type yes or no.")

    if answer == "no":
        print("Cancelled.")
    else:
        print("Running code...")
        # Start timing
        t0 = time.time()
        start_time_str = datetime.now().strftime("Start time: %a %b %d %H:%M:%S %Y")
        print(start_time_str)
        # ----------------------------------------------------------------------------
        pwd = os.getcwd()
        #    
        infile = os.path.join(pwd,"output","01_all_IVI_sorted.txt")
        #
        all_data_file = os.path.join(pwd,"output","01_all_distances.txt")
        # read the data and kip 2 rows due to format error
        data_ini = pd.read_csv(infile, sep=',',skiprows=0,header=0,)
        all_data = pd.read_csv(all_data_file, sep=',',skiprows=0,header=0,)
        # Map species code -> Nomenclature from data_ini
        code_to_name = dict(zip(data_ini['sourcecode'], data_ini['Nomenclature']))
        # data_ini.head(20)
        # Select the data that less than 75% of the cumulative IVI
        data_selected = data_ini.copy()
        # Now filter the data with selected spices 
        all_data_selected = all_data.head(0)
        for i,sourcecodenow in enumerate(data_selected['sourcecode']):
            all_data_now = all_data[all_data['source_code']==sourcecodenow];
            all_data_now.reset_index(drop=True,inplace=True)
            
            all_data_selected = pd.concat([all_data_selected,all_data_now],axis=0)
        all_data_selected.reset_index(drop=True,inplace=True)
        points = all_data_selected[['source_long', 'source_lat']].drop_duplicates().to_numpy()
        # Get unique mapping: tree -> species code
        source_info = all_data_selected[['source', 'source_code']].drop_duplicates()

        # # Area of study area
        # area = area_deg_plain(data['Sourcelong'].min(),data['Sourcelat'].min(),data['Sourcelong'].max(),data['Sourcelat'].max())
        # # radius
        # radius=avg_dist/111320; # in deg
        # #
        dk_dr_values=[];
        radiis=[]
        ref_points_long=[];
        ref_points_lat=[];
        # calculate the average dist
        all_data_selected['pair'] = all_data_selected.apply(lambda row: tuple(sorted([row['source_long'], row['source_lat']])),axis=1)
        all_data_selected_unique = all_data_selected.drop_duplicates(subset='pair')
        avg_dist = all_data_selected_unique['Dist'].mean()
        # radius
        # radius=avg_dist/111320; # in deg
        radius = int(avg_dist);
        area = data_ini['Area'].unique()[0]
        # area = data_ini['Area'].unique()[0]/10000 # convert to hecta
        # Now calculate the Gr for all the data points
        # points_global = all_data_selected[['source_code','source_long', 'source_lat']].drop_duplicates().to_numpy()
        unique_points = all_data_selected[['source', 'source_long', 'source_lat','source2x','source2y']].drop_duplicates()
        # Đổi thành numpy array: mỗi phần tử là (name, (lon, lat))
        points_global = [(row['source'], (row['source_long'], row['source_lat'],row['source2x'],row['source2y'])) for _, row in unique_points.iterrows()]
        #
        points_xy = all_data_selected[['source2x', 'source2y']].drop_duplicates().to_numpy()
        #
        num_cpus = min(40, cpu_count()) # 40 Cpu or max
        print("Calculating GR with critical radius = {} (m), area = {} (m), number of point = {}, using {} CPUs...".format(radius,area,len(points_global),num_cpus))
        # Prepare argument list for parallel pool
        # args_list = [(tuple(point), radius, area, int(avg_dist)+1) for point in points_global]
        args_list = [(point, radius, area, int(avg_dist)+1,points_xy) for point in points_global]
    
        # Parallel computation
        with Pool(processes=num_cpus) as pool:
            results = pool.map(compute_gr_for_point, args_list)
        # for args_list_now in args_list:
        #     # print(args_list_now)
            
        #     results = compute_gr_for_point(args_list_now) 
        # results
        dataout = pd.concat(results, ignore_index=True)
        
        ref_point=[108.5754,14.4991]
        dataout = dist2ref(dataout, ref_point)
        # Merge into dataout (Point == source)
        dataout = dataout.merge(
            source_info,
            left_on='Point',
            right_on='source',
            how='left'
        )
        # Add Nomenclature using code_to_name from data_ini
        dataout['Nomenclature'] = dataout['source_code'].map(code_to_name)
        #
        codes = sorted(data_ini['sourcecode'].unique())
        sp_map = {code: i for i, code in enumerate(codes)}
        dataout['sp_id'] = dataout['source_code'].map(sp_map)
        # # ====================================================================================================================
        dataout.to_csv(os.path.join(pwd,"output","02_species_GR.txt"),index=False,sep=',',encoding='utf-8')
        print(' >>>> Export the output file: {}'.format(os.path.join(pwd,"output","02_species_GR.txt")))
        dataout['radius_m'] = dataout['radius_m'].apply(lambda x: float(f"{x:.1f}"))
        # End timing
        t1 = time.time()
        end_time_str = datetime.now().strftime("End time: %a %b %d %H:%M:%S %Y")  
        total_time = t1 - t0
        print(end_time_str)
        print("Total process time: {:.2f} seconds".format(total_time))
        print("done!")

This code run for 2 spices - so all data will be select. RUN? (yes/no):  yes


Running code...
Start time: Thu Dec 04 15:56:39 2025
Calculating GR with critical radius = 55 (m), area = 13386.634249231713 (m), number of point = 85, using 40 CPUs...
 >>>> Export the output file: /data/longhv/Projects/VietNga_species_distribution_example/spatial_mapping_v2/output/02_species_GR.txt
End time: Thu Dec 04 15:56:42 2025
Total process time: 3.06 seconds
done!


In [26]:
dataout
# all_data_selected

,Point,Longitude,Latitude,radius_m,gr,Lr,distance,d2x,d2y,source,source_code,Nomenclature,sp_id
0,S0417,108.576379,14.499346,1.0,0.000000,-1.000000,108.986463,105.532911,27.218628,S0417,A42,Dacrydium elatum,1
1,S0417,108.576379,14.499346,2.0,0.000000,-2.000000,108.986463,105.532911,27.218628,S0417,A42,Dacrydium elatum,1
2,S0417,108.576379,14.499346,3.0,3.590923,-3.000000,108.986463,105.532911,27.218628,S0417,A42,Dacrydium elatum,1
3,S0417,108.576379,14.499346,4.0,2.693192,2.564379,108.986463,105.532911,27.218628,S0417,A42,Dacrydium elatum,1
4,S0417,108.576379,14.499346,5.0,2.154554,1.564379,108.986463,105.532911,27.218628,S0417,A42,Dacrydium elatum,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4302,S0276,108.575532,14.499508,55.0,0.000000,-55.000000,47.345145,14.272265,45.142721,S0276,A21,Polyosma Sp,0
4303,S0276,108.575532,14.499508,56.0,0.000000,-56.000000,47.345145,14.272265,45.142721,S0276,A21,Polyosma Sp,0
4304,S0276,108.575532,14.499508,57.0,0.000000,-57.000000,47.345145,14.272265,45.142721,S0276,A21,Polyosma Sp,0
4305,S0276,108.575532,14.499508,58.0,0.000000,-58.000000,47.345145,14.272265,45.142721,S0276,A21,Polyosma Sp,0


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pygmt
import os
from scipy.spatial.distance import pdist
# ======================================================================
'''
Plot the figure for each radius distances
'''
# ======================================================================
pwd = os.getcwd();
#
indata = pd.read_csv(os.path.join(pwd,"output","02_species_GR.txt"),sep=",",header=0)
# Output figure
out_fig_dir = os.path.join(pwd,"figures","GR_radius_A42_A72")
os.makedirs(out_fig_dir,exist_ok=True)
# ======================================================================
for i,radius in enumerate(indata['radius_m']):
    print(radius)
    if i > 50:
        break
    # Extract the data
    data_now = indata[indata['radius_m']==radius]
    data_now.reset_index(drop=True,inplace=True)
    
    data_now_1 = data_now[data_now['sp_id']==0]; data_now_1.reset_index(drop=True,inplace=True)
    data_now_2 = data_now[data_now['sp_id']==1]; data_now_2.reset_index(drop=True,inplace=True)
    # Now plot the figure
    fig = pygmt.Figure()
    pygmt.config(FONT_LABEL="14p,Times-Bold,black",
                 FONT_TITLE="15p,Times-Bold,black",
                 FONT_ANNOT_PRIMARY="10p,Times-Bold,black",
                 FONT_ANNOT_SECONDARY="10p,Times-Bold,black"
                )
    area = [0,np.ceil(int(indata['d2x'].max())/10)*10,0,np.ceil(int(indata['d2y'].max())/10)*10]
    # grid = pygmt.datasets.load_earth_relief(resolution="01s", region=area)
    # fig.grdimage(grid=grid, projection="X5i/5i", frame="a", cmap="geo")
    # fig.colorbar(frame=["a100", "x+lElevation", "y+lm"])
    fig.basemap(region=area, projection="X5i/5i", frame=['xafg+l"Easting(m)"','yafg+l"Northing(m)"', 'WSne+t"G(r) map with radius of {} (m)"'.format(radius)])
    gr_grd = pygmt.surface(x=data_now['d2x'], y=data_now['d2y'], z=data_now['gr'],
                            region=area, spacing='0.5/0.5') # 1 metter per 1 metter
    # pygmt.makecpt(cmap=["0   blue","1   white","{}   red".format(np.ceil((data_now['gr'].max())*10)/10)],series=[0, (np.ceil((data_now['gr'].max())*10)/10), 0.01])
    with open("custom.cpt", "w") as f:
        f.write(
            """
            0	blue	1	white
            1	white	{}	red
            B	blue
            F	white
            N	gray
            """.format(int(data_now['gr'].max()))
            )
    # -------------------------------------------------------------------
    # pygmt.makecpt(cmap=cmap,series=[0, 7, 0.01])
    fig.grdimage(grid=gr_grd,
                 # shading=True,
                 # projection="X5i/5i",
                 cmap="custom.cpt")
    
    fig.plot(x=data_now_1['d2x'],y=data_now_1['d2y'],
             style='c0.1c',
             pen='1p,black',
             label=data_now_1['Nomenclature'].unique()
    )
    fig.plot(x=data_now_2['d2x'],y=data_now_2['d2y'],
             style='a0.1c',
             pen='1p,black',
             label=data_now_2['Nomenclature'].unique()
    )
    fig.colorbar(
        cmap="custom.cpt",
        position="JMR+o0.25i/0.0c+w5.0i/0.25c+n",
        box=None,
        frame=["xaf+lG(r)"],
        # scale=1,  # Adjust the scale
        # B="x10"
    )
    fig.legend(position="jBL+o0.5c/0.5c", box="+gwhite@50+p1p")
    fig.savefig(os.path.join(out_fig_dir,"00_distribution_gr_{}_km.png".format(radius)),crop=True, dpi=300, transparent=False)
    # fig.show()
    # ---------------------------------------------------------------------------------------------------------------------------
    fig1 = pygmt.Figure()
    pygmt.config(FONT_LABEL="14p,Times-Bold,black",
                 FONT_TITLE="15p,Times-Bold,black",
                 FONT_ANNOT_PRIMARY="10p,Times-Bold,black",
                 FONT_ANNOT_SECONDARY="10p,Times-Bold,black"
                )
    area = [0,np.ceil(int(indata['d2x'].max())/10)*10,0,np.ceil(int(indata['d2y'].max())/10)*10]
    # grid = pygmt.datasets.load_earth_relief(resolution="01s", region=area)
    # fig.grdimage(grid=grid, projection="X5i/5i", frame="a", cmap="geo")
    # fig.colorbar(frame=["a100", "x+lElevation", "y+lm"])
    fig1.basemap(region=area, projection="X5i/5i", frame=['xafg+l"Easting(m)"','yafg+l"Northing(m)"', 'WSne+t"L(r) map with radius of {} (m)"'.format(radius)])
    gr_grd = pygmt.surface(x=data_now['d2x'], y=data_now['d2y'], z=data_now['Lr'],
                            region=area, spacing='0.5/0.5') # 1 metter per 1 metter
    # pygmt.makecpt(cmap=["0   blue","1   white","{}   red".format(np.ceil((data_now['gr'].max())*10)/10)],series=[0, (np.ceil((data_now['gr'].max())*10)/10), 0.01])
    with open("custom_Kr.cpt", "w") as f:
        f.write(
            """
            {}	blue	0	white
            0	white	{}	red
            B	blue
            F	white
            N	gray
            """.format((data_now['Lr'].min()),(data_now['Lr'].max()))
            )
    # -------------------------------------------------------------------
    # pygmt.makecpt(cmap=cmap,series=[0, 7, 0.01])
    fig1.grdimage(grid=gr_grd,
                 # shading=True,
                 # projection="X5i/5i",
                 cmap="custom_Kr.cpt")
    
    fig1.plot(x=data_now_1['d2x'],y=data_now_1['d2y'],
             style='c0.1c',
             pen='1p,black',
             label=data_now_1['Nomenclature'].unique()
    )
    fig1.plot(x=data_now_2['d2x'],y=data_now_2['d2y'],
             style='a0.1c',
             pen='1p,black',
             label=data_now_2['Nomenclature'].unique()
    )
    
    fig1.colorbar(
        cmap="custom_Kr.cpt",
        position="JMR+o0.25i/0.0c+w5.0i/0.25c+n",
        box=None,
        frame=["xaf+lL(r)"],
        # scale=1,  # Adjust the scale
        # B="x10"
    )
    fig1.legend(position="jBL+o0.5c/0.5c", box="+gwhite@50+p1p")
    fig1.savefig(os.path.join(out_fig_dir,"01_distribution_Lr_{}_km.png".format(radius)),crop=True, dpi=300, transparent=False)
    
print("Done!")

surface [WARNING]: 3 unusable points were supplied; these will be ignored.
surface [WARNING]: You should have pre-processed the data with block-mean, -median, or -mode.
surface [WARNING]: Check that previous processing steps write results with enough decimals.
surface [WARNING]: Possibly some data were half-way between nodes and subject to IEEE 754 rounding.


1.0


surface [WARNING]: 3 unusable points were supplied; these will be ignored.
surface [WARNING]: You should have pre-processed the data with block-mean, -median, or -mode.
surface [WARNING]: Check that previous processing steps write results with enough decimals.
surface [WARNING]: Possibly some data were half-way between nodes and subject to IEEE 754 rounding.
surface [WARNING]: 3 unusable points were supplied; these will be ignored.
surface [WARNING]: You should have pre-processed the data with block-mean, -median, or -mode.
surface [WARNING]: Check that previous processing steps write results with enough decimals.
surface [WARNING]: Possibly some data were half-way between nodes and subject to IEEE 754 rounding.


2.0


surface [WARNING]: 3 unusable points were supplied; these will be ignored.
surface [WARNING]: You should have pre-processed the data with block-mean, -median, or -mode.
surface [WARNING]: Check that previous processing steps write results with enough decimals.
surface [WARNING]: Possibly some data were half-way between nodes and subject to IEEE 754 rounding.
surface [WARNING]: 3 unusable points were supplied; these will be ignored.
surface [WARNING]: You should have pre-processed the data with block-mean, -median, or -mode.
surface [WARNING]: Check that previous processing steps write results with enough decimals.
surface [WARNING]: Possibly some data were half-way between nodes and subject to IEEE 754 rounding.


...

50.0


surface [WARNING]: 3 unusable points were supplied; these will be ignored.
surface [WARNING]: You should have pre-processed the data with block-mean, -median, or -mode.
surface [WARNING]: Check that previous processing steps write results with enough decimals.
surface [WARNING]: Possibly some data were half-way between nodes and subject to IEEE 754 rounding.
surface [WARNING]: Input data lie exactly on a plane.
grdimage [ERROR]: Z-slice around line 1 with dz <= 0
[Session pygmt-session (1891)]: Error returned from GMT API: GMT_CPT_READ_ERROR (8)
[Session pygmt-session (1891)]: Error returned from GMT API: GMT_CPT_READ_ERROR (8)
grdimage [ERROR]: Unable to save current CPT file to /home/longhv/.gmt/sessions/gmt_session.56063/gmt.100.cpt !
grdimage [ERROR]: Failed to read CPT custom_Kr.cpt.
colorbar [ERROR]: Z-slice around line 1 with dz <= 0
[Session pygmt-session (1897)]: Error returned from GMT API: GMT_CPT_READ_ERROR (8)
[Session pygmt-session (1897)]: Error returned from GMT API: GM

51.0


surface [WARNING]: 3 unusable points were supplied; these will be ignored.
surface [WARNING]: You should have pre-processed the data with block-mean, -median, or -mode.
surface [WARNING]: Check that previous processing steps write results with enough decimals.
surface [WARNING]: Possibly some data were half-way between nodes and subject to IEEE 754 rounding.
surface [WARNING]: Input data lie exactly on a plane.
grdimage [ERROR]: Z-slice around line 1 with dz <= 0
[Session pygmt-session (1929)]: Error returned from GMT API: GMT_CPT_READ_ERROR (8)
[Session pygmt-session (1929)]: Error returned from GMT API: GMT_CPT_READ_ERROR (8)
grdimage [ERROR]: Unable to save current CPT file to /home/longhv/.gmt/sessions/gmt_session.56063/gmt.102.cpt !
grdimage [ERROR]: Failed to read CPT custom_Kr.cpt.
colorbar [ERROR]: Z-slice around line 1 with dz <= 0
[Session pygmt-session (1935)]: Error returned from GMT API: GMT_CPT_READ_ERROR (8)
[Session pygmt-session (1935)]: Error returned from GMT API: GM

52.0
Done!


In [4]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pygmt
import os
from scipy.spatial.distance import pdist
# ======================================================================
'''
Plot the figure for each radius distances
'''
# ======================================================================
pwd = os.getcwd();
#
indata = pd.read_csv(os.path.join(pwd,"output","02_species_GR.txt"),sep=",",header=0)
# Output figure
out_fig_dir = os.path.join(pwd,"figures","GR_Lr_tree_A42_A72")
os.makedirs(out_fig_dir,exist_ok=True)
# ======================================================================
# Extract each point data
for i,point in enumerate(indata['Point'].unique()):
    print(point)
    if i > len(indata['Point'].unique()):
        break
    # if i != 2: 
    #     continue
    datanow = indata[indata['Point']==point];
    datanow.reset_index(drop=True,inplace=True)
    # ------------------------------------------------------------------
    fig = pygmt.Figure()
    pygmt.config(FONT_LABEL="14p,Times-Bold,black",
                 FONT_TITLE="15p,Times-Bold,black",
                 FONT_ANNOT_PRIMARY="10p,Times-Bold,black",
                 FONT_ANNOT_SECONDARY="10p,Times-Bold,black"
                )
    area = [0,int(datanow['radius_m'].max()),0,datanow['gr'].max()+1]
    fig.basemap(region=area, projection="X5i/2.5i", frame=['xafg+l"Radius(m)"','yafg+l"G(r)"', 'WSne+t"G(r) for tree {}"'.format(point)])
    fig.plot(x=datanow['radius_m'],y=datanow['gr'],pen="1.5p,black")
    fig.plot(x=datanow['radius_m'],y=datanow['gr'],style="c0.25c",pen="0.5p,black",fill="red")
    fig.plot(x=[0,1000],y=[1,1],pen="1.0p,purple,-")
    #
    fig.shift_origin(xshift="+w1c")
    # area1 = [0,int(datanow['radius_m'].max()),datanow['Lr'].min()-1,datanow['Lr'].max()+1]
    area1 = [0,int(datanow['radius_m'].max()),-10,datanow['Lr'].max()+1]
    fig.basemap(region=area1, projection="X5i/2.5i", frame=['xafg+l"Radius(m)"','yafg+l"L(r)"', 'wSnE+t"L(r) for tree {}"'.format(point)])
    fig.plot(x=datanow['radius_m'],y=datanow['Lr'],pen="1.5p,black")
    fig.plot(x=datanow['radius_m'],y=datanow['Lr'],style="c0.25c",pen="0.5p,black",fill="blue")
    fig.plot(x=[0,1000],y=[0,0],pen="1.0p,purple,-")
    fig.savefig(os.path.join(out_fig_dir,"00_{}.png".format(point)),crop=True, dpi=300, transparent=False)
    # fig.show()
print("Done!")

S0417
S0418
S0419
S0420
S0421
S0422
S0423
S0424
S0425
S0426
S0427
S0428
S0429
S0430
S0431
S0432
S0433
S0434
S0435
S0436
S0437
S0438
S0439
S0440
S0441
S0442
S0443
S0444
S0445
S0446
S0447
S0448
S0449
S0450
S0451
S0452
S0453
S0454
S0621
S0622
S0623
S0624
S0625
S0626
S0627
S0628
S0629
S0630
S0631
S0632
S0633
S0634
S0635
S0636
S0637
S0638
S0639
S0640
S0641
S0642
S0643
S0644
S0645
S0646
S0647
S0648
S0649
S0650
S0651
S0652
S0653
S0654
S0655
S0656
S0657
S0658
S0659
S0660
S0661
S0662
S0663
S0664
S0665
S0758
S0759
Done!
